[View in Colaboratory](https://colab.research.google.com/github/mrdeds/deep-learning-coursera/blob/master/frecuencia_de_n_grams.ipynb)

In [1]:
!pip install psycopg2
!pip install stop_words

In [2]:
import psycopg2
import pandas as pd
import json
from nltk import everygrams, FreqDist
from nltk.tokenize import RegexpTokenizer
import numpy as np

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving creds_data.json to creds_data (1).json
User uploaded file "creds_data.json" with length 273 bytes


In [0]:
creds = uploaded.get('creds_data.json')
dic_creds = json.loads(creds.decode())

In [0]:
host_check=fn

host_wh=dic_creds.get('host_wh')
port_wh=dic_creds.get('port_wh')
user_wh=dic_creds.get('user_wh')
password_wh=dic_creds.get('password_wh')
database_wh=dic_creds.get('database_wh')

host_l=dic_creds.get('host_l')
port_l=dic_creds.get('port_l')
user_l=dic_creds.get('user_l')
password_l=dic_creds.get('password_l')
database_l=dic_creds.get('database_l')

In [0]:
def sql_to_dataframe(query, base='wh'):
  if base=='wh':
    conn = psycopg2.connect(host=host_wh, port=port_wh, user=user_wh, password=password_wh, database=database_wh,)
  elif base=='llamadas':
    conn = psycopg2.connect(host=host_l, port=port_l, user=user_l, password=password_l, database=database_l,)
  try:
    cursor = conn.cursor()
    cursor.execute(query)
    DataSQL = cursor.fetchall() # careful, the results could be huge
    conn.commit()
  finally:
    conn.close()
  
  colnames = [desc[0] for desc in cursor.description]
  DF = pd.DataFrame(DataSQL) # Ponemos datos en DataFrame
  DF.columns = colnames # Asignamos nombres a las columnas
  
  return DF

In [0]:
query = "SELECT rh.id AS \"ID RH\",\
    rh.name AS \"Nombre del RH\",\
    rh.numeroextension__c AS \"Número de extensión\",\
    RIGHT(lead.phone,8) AS \"phone\",\
    lead.id AS \"ID Lead\",\
    lead.rating as \"Calificación\"\
    FROM rh_all_recursoshumanos__c AS rh\
    JOIN salesforce_lead AS lead ON rh.ID=lead.vendedorasignado__c\
    WHERE rh.name not in ('Agente de Preventa','ARD Virtual') and phone not in ('','None')\
    GROUP BY rh.name, rh.id, lead.phone, rh.numeroextension__c, lead.id, lead.rating\
    ORDER BY rh.id ASC,\
    rh.numeroextension__c ASC"

salesforce_DF = sql_to_dataframe(query)

In [8]:
salesforce_DF.head(10)
len(salesforce_DF)

221371

In [9]:
type(salesforce_DF)

pandas.core.frame.DataFrame

In [0]:
query = "SELECT rh.id AS \"ID RH\",\
    rh.name AS \"Nombre del RH\",\
    rh.numeroextension__c AS \"Número de extensión\",\
    RIGHT(lead.mobilephone,8) AS \"phone\",\
    lead.id AS \"ID Lead\",\
    lead.rating as \"Calificación\"\
    FROM rh_all_recursoshumanos__c AS rh\
    JOIN salesforce_lead AS lead ON rh.ID=lead.vendedorasignado__c\
    WHERE rh.name not in ('Agente de Preventa','ARD Virtual') and phone not in ('','None')\
    GROUP BY rh.name, rh.id, lead.mobilephone, rh.numeroextension__c, lead.id, lead.rating\
    ORDER BY rh.id ASC,\
    rh.numeroextension__c ASC"

salesforce_mobile_DF = sql_to_dataframe(query)

In [0]:
len(salesforce_mobile_DF)

221083

In [0]:
query_llamadas = "SELECT id as \"id de llamada\",\
texto,\
llama AS extension,\
RIGHT(recibe,8) as \"phone\" \
FROM llamadas \
where texto != '' \
GROUP BY id, texto, llama, recibe \
ORDER BY id asc, texto, llama, recibe"

llamadas_DF = sql_to_dataframe(query_llamadas, base="llamadas")

In [0]:
llamadas_DF.head(10)
len(llamadas_DF.index)

25932

In [0]:
DF_fijo = pd.merge(salesforce_DF,llamadas_DF, on='phone')

In [0]:
DF_mobile = pd.merge(salesforce_mobile_DF, llamadas_DF, on='phone')

In [0]:
DF = pd.concat([DF_mobile, DF_fijo])

In [0]:
len(DF)

11419

In [0]:
Cerrados = DF.loc[DF['calificación'] == 'Cerrado']
Cerrados_fijo = DF_fijo.loc[DF_fijo['calificación'] == 'Cerrado']
Cerrados_mobile = DF_mobile.loc[DF_mobile['calificación'] == 'Cerrado']

In [0]:
ratio = len(Cerrados)/len(DF)
ratio_fijo = len(Cerrados_fijo)/len(DF_fijo)
ratio_mobile = len(Cerrados_mobile)/len(DF_mobile)

print('ratio de cerrados a no cerrados total: {}'.format(ratio))
print('ratio de cerrados a no cerrados fijo: {}'.format(ratio_fijo))
print('ratio de cerrados a no cerrados mobile: {}'.format(ratio_mobile))
print('leads totales:{}'.format(len(DF_mobile)))

ratio de cerrados a no cerrados total: 0.5387512041334618
ratio de cerrados a no cerrados fijo: 0.5108079104706423
ratio de cerrados a no cerrados mobile: 0.5759803921568627
leads totales:4896


In [0]:
tokenizer = RegexpTokenizer('[A-zÀ-ÿ]\w+')
textos = DF["texto"]
DF["tokens"] = DF["texto"].apply(tokenizer.tokenize)

In [0]:
from stop_words import get_stop_words

stop_words = get_stop_words('spanish')

In [0]:
X_prev = DF['tokens'].values
lista_frase_limpia = []
lista_texto_limpio = []
for lista in X_prev:
    frase_limpia = [palabra for palabra in lista if palabra not in stop_words]
    texto_limpio =  ' '.join(frase_limpia)
    lista_frase_limpia.append(frase_limpia)

    lista_texto_limpio.append(texto_limpio)

DF['clean tokens'] = lista_texto_limpio
DF['clean text'] = lista_frase_limpia


In [0]:
lista_ngrams = []

for textos in DF['clean text']:
  ngrams = list(everygrams(textos, 3, 6))
  lista_ngrams.append(ngrams)

In [0]:
DF['ngrams'] = lista_ngrams

In [0]:
DF=DF.drop(DF[DF.ngrams.map(len) <= 2].index)

In [0]:
n_grams = DF[(DF.calificación == 'Cerrado') & (DF.ngrams != '[]')]["ngrams"]

In [0]:
todos_ngrams = [ngram for ngrams in lista_ngrams for ngram in ngrams]

In [0]:
todos_ngrams[0]

('Hola', 'buen', 'día')

In [0]:
from collections import Counter

counter=Counter(todos_ngrams)

kk=[list(counter.keys()),list(counter.values())]

freq_ngrams = pd.DataFrame(np.array(kk).T, columns=['N-Gram','Count']).sort_values(by=['Count'] , ascending=False)

In [0]:
freq_ngrams

,N-Gram,Count
767,"(Hola, buenas, tardes)",1054
47672,"(segundo, Por, favor)",915
178660,"(ayer, tocaba, hacer)",850
12590,"(mensaje, después, tono)",743
33378,"(bueno, bueno, bueno)",714
17672,"(si, alguna, duda)",669
12587,"(usted, puede, dejar)",608
12588,"(puede, dejar, mensaje)",589
12598,"(usted, puede, dejar, mensaje)",558
870,"(resuelve, deuda, Cómo)",558
